In [ ]:
#!pip install tqdm

In [68]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D, BatchNormalization
from keras.applications import ResNet50
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import os, sys
import copy
from PIL import Image
from PIL import ImageFilter
import shutil
import glob
from PIL import ImageEnhance
import subprocess as sbp
import os.path
from keras.models import Sequential,Model,load_model
import keras_metrics

#import fastai
#from fastai.vision import *

#import matplotlib.pyplot as plt
#%matplotlib inline

#import matplotlib.pyplot as plt
#%matplotlib inline






trainLabelsPath = "traininglabels.csv"
trainImagesPath = "/Users/jdaya44/Documents/DevMasters/WiDS Project/train_images_15244/"

data_aug = "/Users/jdaya44/Documents/DevMasters/WiDS Project/train_images_15244/data_aug"
os.makedirs(data_aug)

testFolderPath = "/Users/jdaya44/Documents/DevMasters/WiDS Project/leaderboard_test_data"
testImagesPath = "/Users/jdaya44/Documents/DevMasters/WiDS Project/leaderboard_test_data/"

holdFolderPath = "/Users/jdaya44/Documents/DevMasters/WiDS Project/leaderboard_holdout_data"
holdImagesPath = "/Users/jdaya44/Documents/DevMasters/WiDS Project/leaderboard_holdout_data/"

subFilePath = "finalSubmission_hybridScore.csv"



In [72]:
#data_aug = "/Users/jdaya44/Documents/DevMasters/WiDS Project/train_images_15244/data_aug"
#shutil.rmtree(data_aug)
#os.makedirs(data_aug)

In [ ]:

trainLabelsPath = "/home/ubuntu/WIDS/traininglabels.csv"
trainImagesPath = "/home/ubuntu/WIDS/train_images/"

data_aug = "/home/ubuntu/WIDS/train_images/data_aug"
shutil.rmtree(data_aug)
os.makedirs(data_aug)


testFolderPath = "/home/ubuntu/WIDS/leaderboard_test_data"
testImagesPath = "/home/ubuntu/WIDS/leaderboard_test_data/"

holdFolderPath = "/home/ubuntu/WIDS/leaderboard_holdout_data"
holdImagesPath = "/home/ubuntu/WIDS/leaderboard_holdout_data/"

subFilePath = "finalSubmission_hybridScore.csv"

In [73]:
data = pd.read_csv(trainLabelsPath)

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15244 entries, 0 to 15243
Data columns (total 3 columns):
image_id       15244 non-null object
has_oilpalm    15244 non-null int64
score          15244 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 357.4+ KB


In [75]:
data_yes = data[(data.has_oilpalm == 1) &(data.score == 1)]

data_no = data[(data.has_oilpalm == 0) & (data.score == 1)]

In [76]:
data = data_yes.append(data_no)

In [77]:
data.shape

(12723, 3)

In [78]:
data = data.sort_values(by='image_id').reset_index(drop=True)

In [79]:
x = data
y = data.has_oilpalm

X_train, X_test, y_train, y_test = train_test_split(x,y, random_state = 42, test_size=0.2)

X_train = X_train.reset_index(drop =True)
y_train = y_train.reset_index(drop=True)

# Create Variables

In [80]:
X_train.shape

(10178, 3)

In [81]:
X_train.nunique()

image_id       10178
has_oilpalm        2
score              1
dtype: int64

In [82]:
X_train.has_oilpalm.value_counts()

0    9567
1     611
Name: has_oilpalm, dtype: int64

In [83]:
X_train[X_train.has_oilpalm==1]

,image_id,has_oilpalm,score
43,img_064952018.jpg,1,1.0
78,img_047702017.jpg,1,1.0
85,img_103712018.jpg,1,1.0
98,img_032392018.jpg,1,1.0
108,img_031602018.jpg,1,1.0
112,img_025812017.jpg,1,1.0
130,img_006522017.jpg,1,1.0
143,img_008132017.jpg,1,1.0
152,img_068672018.jpg,1,1.0
197,img_034882017.jpg,1,1.0


In [84]:
rotate180=[]
rotate90 =[]
rotate270 = []
imblur=[]
imunsharp = []
rr =[]
gg= []
bb = []
ll = []
colorr = []
maxfilter = []
minfilter=[]
colornew=[]
sharpp=[]
sharpa=[]
X_train_images = []




for i in tqdm(range(len(X_train))):
    #img = Image.open('/Users/jdaya44/Documents/DevMasters/WiDS Project/train_images_15244/'+ X_train.image_id[i])
    #X_train_images.append(img)
    if X_train.has_oilpalm[i] ==1:
        im = Image.open(trainImagesPath + X_train.image_id[i])
        rotate_180 = im.rotate(180) 
        rotate_180.save(data_aug + '/' + "180_" + X_train.image_id[i])
        rotate_90 = im.rotate(90)
        rotate_90.save(data_aug + '/' + "90_" + X_train.image_id[i])
        rotate_270 = im.rotate(270)
        rotate_270.save(data_aug + '/' + '270_' + X_train.image_id[i])
        im_blur=im.filter(ImageFilter.GaussianBlur)
        im_blur.save(data_aug + '/' + "blur_" + X_train.image_id[i])
        im_unsharp=im.filter(ImageFilter.UnsharpMask)
        im_unsharp.save(data_aug + '/' + 'unsharp_' +  X_train.image_id[i])
        im.convert('RGB')
        r,g,b=im.split()
        r=r.convert("RGB")
        r.save(data_aug + '/' +'r_' + X_train.image_id[i])
        im.convert('RGB')
        r,g,b=im.split()
        g=g.convert("RGB")
        g.save(data_aug + '/' + 'g_' + X_train.image_id[i])
        im.convert('RGB')
        r,g,b=im.split()
        b=b.convert("RGB")
        b.save(data_aug + '/' + 'b_' + X_train.image_id[i])
        l = im.convert('L')
        l.save(data_aug + '/' + 'L_' + X_train.image_id[i])
        enhancer = ImageEnhance.Color(im)
        factor = 0.75
        color = enhancer.enhance(factor)
        color.save(data_aug + '/' + 'color_' + X_train.image_id[i])
        max_filter = im.filter(ImageFilter.MaxFilter)
        max_filter.save(data_aug + '/' + 'max_' + X_train.image_id[i])
        min_filter = im.filter(ImageFilter.MinFilter)
        min_filter.save(data_aug + '/' + 'min_'+ X_train.image_id[i])
        enhancer_new = ImageEnhance.Color(im)
        factor_new = 0.25
        color_new = enhancer_new.enhance(factor_new)
        color_new.save(data_aug + '/' + 'sat_' + X_train.image_id[i])
        enhancer = ImageEnhance.Sharpness(im)
        factor = 1.25
        sharp = enhancer.enhance(factor)
        sharp.save(data_aug + '/' + 'sharp_' + X_train.image_id[i])
        enhancer_a = ImageEnhance.Sharpness(im)
        factor_a = 1.75
        sharp_a = enhancer_a.enhance(factor_a)
        sharp_a.save(data_aug + '/' + 'sharpness_' + X_train.image_id[i])
    
        
        
        
        
        


100%|██████████| 10178/10178 [01:00<00:00, 168.47it/s]


In [85]:
imageList = os.listdir(data_aug)
imageList

['sharp_img_074792018.jpg',
 'sharpness_img_057332017.jpg',
 'color_img_018122018.jpg',
 'min_img_052922017.jpg',
 'g_img_014042018.jpg',
 'L_img_029602017.jpg',
 'color_img_068542018.jpg',
 '180_img_012512017.jpg',
 'color_img_004812018.jpg',
 'b_img_104182017.jpg',
 '90_img_013692017.jpg',
 'sharp_img_075012018.jpg',
 'max_img_013312017.jpg',
 'sharp_img_069922018.jpg',
 'sharpness_img_064492017.jpg',
 'g_img_101502017.jpg',
 '180_img_044672017.jpg',
 '270_img_104442017.jpg',
 'sharp_img_061632018.jpg',
 'sharp_img_076262017.jpg',
 'sat_img_107952018.jpg',
 'color_img_091192017.jpg',
 'unsharp_img_089942018.jpg',
 '90_img_062472017.jpg',
 'color_img_058192017.jpg',
 '180_img_014042017.jpg',
 'g_img_023742017.jpg',
 'max_img_100202018.jpg',
 'b_img_066372018.jpg',
 '180_img_088152018.jpg',
 'min_img_097692018.jpg',
 'color_img_079542017.jpg',
 'sharpness_img_060532018.jpg',
 'sharp_img_017702018.jpg',
 'blur_img_006162017.jpg',
 'g_img_005042017.jpg',
 'sharpness_img_090972017.jpg',
 

In [86]:
images = pd.DataFrame({'image_id': imageList, 'has_oilpalm': 1})

In [87]:
#images.sort_values(by = 'image_id')

In [88]:
#X_train.sort_values(by = 'image_id')

In [89]:
#df_final = images.append(X_train)

In [90]:
#df_final.info()

In [91]:
#df_final = df_final.drop('score', axis=1)

In [92]:
#df_final = df_final.reset_index(drop=True)

In [93]:
#df_final.has_oilpalm.value_counts()

In [94]:
#df_final.head()

In [95]:
#X = df_final.image_id
#y = df_final.has_oilpalm

#y = to_categorical(y)

In [96]:
#y.shape

In [97]:
#X.shape

In [98]:
#df_final.shape

In [99]:
#type(df_final)

In [100]:
#type(df_final.image_id)

In [101]:
X_test = X_test.drop('has_oilpalm', axis=1)

In [102]:
X_test = X_test.reset_index(drop=True)

In [103]:
train_image = []
X_train_id = []
y_train_label = []
for i in tqdm(range(len(images))):
    X_train_id.append(images.image_id[i])
    y_train_label.append(images.has_oilpalm[i])
    img = image.load_img(data_aug + '/'+ images.image_id[i], target_size=(150,150,1))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
dF_aug = np.array(train_image)
dF_aug

100%|██████████| 9165/9165 [00:23<00:00, 384.93it/s]


array([[[[0.22352941, 0.28627452, 0.18431373],
         [0.22745098, 0.2901961 , 0.1882353 ],
         [0.2627451 , 0.31764707, 0.21960784],
         ...,
         [0.23137255, 0.26666668, 0.18431373],
         [0.23529412, 0.27058825, 0.1882353 ],
         [0.2627451 , 0.29803923, 0.21568628]],

        [[0.19607843, 0.25882354, 0.15686275],
         [0.2       , 0.25490198, 0.15686275],
         [0.22352941, 0.2784314 , 0.18039216],
         ...,
         [0.23921569, 0.27450982, 0.19215687],
         [0.22745098, 0.2627451 , 0.18039216],
         [0.23137255, 0.26666668, 0.18431373]],

        [[0.23137255, 0.28627452, 0.1882353 ],
         [0.21960784, 0.27450982, 0.1764706 ],
         [0.21176471, 0.25882354, 0.16470589],
         ...,
         [0.25882354, 0.29411766, 0.21176471],
         [0.23529412, 0.27058825, 0.1882353 ],
         [0.22352941, 0.25882354, 0.1764706 ]],

        ...,

        [[0.20784314, 0.24313726, 0.16078432],
         [0.24313726, 0.27450982, 0.18431373]

In [104]:
#train_image = []
#X_train_id = []
#y_train_label = []
for i in tqdm(range(len(X_train))):
    X_train_id.append(X_train.image_id[i])
    y_train_label.append(X_train.has_oilpalm[i])
    img = image.load_img(trainImagesPath + X_train.image_id[i], target_size=(150,150,1))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
dF = np.array(train_image)
dF

100%|██████████| 10178/10178 [00:31<00:00, 323.46it/s]


array([[[[0.22352941, 0.28627452, 0.18431373],
         [0.22745098, 0.2901961 , 0.1882353 ],
         [0.2627451 , 0.31764707, 0.21960784],
         ...,
         [0.23137255, 0.26666668, 0.18431373],
         [0.23529412, 0.27058825, 0.1882353 ],
         [0.2627451 , 0.29803923, 0.21568628]],

        [[0.19607843, 0.25882354, 0.15686275],
         [0.2       , 0.25490198, 0.15686275],
         [0.22352941, 0.2784314 , 0.18039216],
         ...,
         [0.23921569, 0.27450982, 0.19215687],
         [0.22745098, 0.2627451 , 0.18039216],
         [0.23137255, 0.26666668, 0.18431373]],

        [[0.23137255, 0.28627452, 0.1882353 ],
         [0.21960784, 0.27450982, 0.1764706 ],
         [0.21176471, 0.25882354, 0.16470589],
         ...,
         [0.25882354, 0.29411766, 0.21176471],
         [0.23529412, 0.27058825, 0.1882353 ],
         [0.22352941, 0.25882354, 0.1764706 ]],

        ...,

        [[0.20784314, 0.24313726, 0.16078432],
         [0.24313726, 0.27450982, 0.18431373]

In [105]:
len(dF)

19343

In [106]:
test_image = []
for i in tqdm(range(len(X_test))):
    img = image.load_img(trainImagesPath+ X_test.image_id[i], target_size=(150,150,1))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X_test = np.array(test_image)
X_test


100%|██████████| 2545/2545 [00:12<00:00, 203.92it/s]


array([[[[0.10196079, 0.16078432, 0.07058824],
         [0.10980392, 0.16862746, 0.07843138],
         [0.07450981, 0.13333334, 0.05098039],
         ...,
         [0.05882353, 0.11372549, 0.04705882],
         [0.07058824, 0.12941177, 0.05490196],
         [0.04705882, 0.10588235, 0.03137255]],

        [[0.09803922, 0.15686275, 0.06666667],
         [0.10196079, 0.16078432, 0.07843138],
         [0.06666667, 0.1254902 , 0.04313726],
         ...,
         [0.09803922, 0.15294118, 0.08627451],
         [0.03529412, 0.09019608, 0.02352941],
         [0.09019608, 0.14117648, 0.07058824]],

        [[0.11372549, 0.17254902, 0.09019608],
         [0.08235294, 0.14117648, 0.05882353],
         [0.09019608, 0.14901961, 0.06666667],
         ...,
         [0.05490196, 0.10588235, 0.03921569],
         [0.02352941, 0.07450981, 0.00784314],
         [0.10980392, 0.16078432, 0.09411765]],

        ...,

        [[0.10196079, 0.17254902, 0.08627451],
         [0.07843138, 0.13725491, 0.05490196]

In [107]:
#dF.shape

model = ResNet50(include_top=False, weights='imagenet', input_shape= (150,150,3))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())

#model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(150,150,3)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(128, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.5))
#model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

#model.layers[0].trainable = False

In [108]:
model = Sequential()

model.add(ResNet50(include_top=False, weights='imagenet', input_shape= (150,150,3)))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [109]:
for layer in model.layers[:60]:
    layer.trainable = False

In [110]:
x = model.output
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dense(264, activation="relu")(x)

prediction = Dense(1, activation="sigmoid")(x)

In [111]:
model_final = Model(input=model.input, output = prediction)

/Users/jdaya44/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("re..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [112]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model_final.compile(optimizer=adam, loss='binary_crossentropy',metrics = [keras_metrics.precision(), keras_metrics.recall(),'accuracy'])

In [113]:
#!pip install keras-metrics

In [114]:

#model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=[keras_metrics.precision(), keras_metrics.recall(),'accuracy'])




In [115]:
y_train_label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [116]:
X_train = dF
y_train = np.array(y_train_label)
y_test = np.array(y_test)

In [117]:
type(y_train)

numpy.ndarray

In [118]:
type(y)

pandas.core.series.Series

In [ ]:
# Build model with training data and get loss and accuracy scores
model_final.fit(dF, y_train, epochs=50, validation_data=(X_test, y_test))

Train on 19343 samples, validate on 2545 samples
Epoch 1/50
19328/19343 [============================>.] - ETA: 1s - loss: 0.0349 - precision: 0.9860 - recall: 0.9877 - acc: 0.9867

In [60]:
#confusion matrix
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import classification_report

#y_pred = model_final.predict(X_test)



KeyboardInterrupt: 

In [ ]:
#y_pred.shape

In [ ]:
#y_pred

In [61]:
testimagesList = sorted(os.listdir(testFolderPath))
testimagesList

NameError: name 'testpath' is not defined

In [ ]:
leaderboard_test_pred = pd.DataFrame(testimagesList , columns=['image_id'])
leaderboard_test_pred.shape

In [ ]:
leaderboard_test_pred.reset_index(drop=True, inplace =True)

In [ ]:
test_image = []
for i in tqdm(range(len(testimagesList))):
    img = image.load_img(testImagesPath+testimagesList[i], target_size=(150,150,1))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)
test

In [ ]:
# making predictions
prediction = model_final.predict(test)

In [ ]:
leaderboard_test_pred['has_oilpalm'] = prediction

leaderboard_test_pred['has_oilpalm']

In [ ]:
holdimagesList = sorted(os.listdir(holdFolderPath))
holdimagesList

In [ ]:
leaderboard_hold_pred = pd.DataFrame(holdimagesList , columns=['image_id'])
len(leaderboard_hold_pred)

In [ ]:


hold_image = []
for i in tqdm(range(len(holdimagesList))):
    img = image.load_img(holdImagesPath+holdimagesList[i], target_size=(150,150,1))
    img = image.img_to_array(img)
    img = img/255
    hold_image.append(img)
hold = np.array(hold_image)
hold

In [ ]:
# making predictions
prediction = model_final.predict(hold)

In [ ]:
leaderboard_hold_pred['has_oilpalm'] = prediction
leaderboard_hold_pred

In [ ]:
leaderboard_test_pred.shape

In [ ]:
leaderboard_hold_pred.shape

In [ ]:
finalSubmission = leaderboard_test_pred.append(leaderboard_hold_pred)
finalSubmission.shape

In [ ]:
finalSubmission.head()

In [ ]:
finalSubmission.has_oilpalm.value_counts()

In [ ]:
finalSubmission.to_csv(subFilePath,index=False)